In [2]:
#Python specific packages
import pandas as pd
import numpy as np
import sys

In [11]:
#specifying local system path for remaining functions files
sys.path.insert(1, '/Users/Cylita/Desktop/insight-ds-project_news-link/scripts')

In [55]:
#self written functions and scripts that will be needed
import text_normalization_funs as TN

In [56]:
import similiarity_funcs as SIM

In [14]:
#Loading in required raw data for processing, dropping all rows that contain NAs
newsJan20 = pd.read_csv('/Users/Cylita/Desktop/insight-ds-project_news-link/data/raw/Jan_20_news.csv').dropna(subset=['title', 'maintext'])
newsJan21 = pd.read_csv('/Users/Cylita/Desktop/insight-ds-project_news-link/data/raw/Jan_21_news.csv').dropna(subset=['title', 'maintext'])
newsJan23 = pd.read_csv('/Users/Cylita/Desktop/insight-ds-project_news-link/data/raw/Jan_23_news.csv').dropna(subset=['title', 'maintext'])
newsJan24 = pd.read_csv('/Users/Cylita/Desktop/insight-ds-project_news-link/data/raw/Jan_24_news.csv').dropna(subset=['title', 'maintext'])
newsJan26 = pd.read_csv('/Users/Cylita/Desktop/insight-ds-project_news-link/data/raw/Jan_26_news.csv').dropna(subset=['title', 'maintext'])
newsJan27 = pd.read_csv('/Users/Cylita/Desktop/insight-ds-project_news-link/data/raw/Jan_27_news.csv').dropna(subset=['title', 'maintext'])

In [34]:
#Concatinating all dataframes
combine_news = pd.concat([newsJan20, newsJan21, newsJan23, newsJan24, newsJan26, newsJan27], axis=0)

In [38]:
#Likely some duplicate stories references over different days and different landing pages
#Removing said duplicates
unique_news = combine_news.drop_duplicates(subset = 'mainurl')
len(unique_news)

1221

In [77]:
#Isolating individual similar and realted links, storying them in separate columns
splitlinks = unique_news['simlinks'].str.split(' ', expand=True)
splitlinks_reduced = splitlinks.loc[:,1:4]
#Some articles have a lot of links, reducing this to only take the first 4 similar links included
splitlinks_foradding = splitlinks_reduced.rename(columns={1:"sim1", 2:'sim2', 3:'sim3', 4:'sim4'})


In [1]:
#Adding the first four similar links as four new columns to the exisiting dataframe
UniqLinkNews = pd.concat([unique_news, splitlinks_foradding], axis=1)

#Export as a newdataframe
#Following dataframe will be used to scrape similar link articles using webscraping functions
UniqLinkNews.to_csv('Cleaned_Split_RawNews.csv')

NameError: name 'pd' is not defined

In [2]:
#Reading in data frame with scraped web articles
sim_news = pd.read_csv("/Users/Cylita/Desktop/insight-ds-project_news-link/data/processed/News_Raw_ValData.csv").dropna(subset=['title', 'maintext'])
trimmed_sim = sim_news[['author', 'date','title','maintext', 'mainurl']]

In [6]:
#Reading in the original dataframe where all similar links were split into columns
UniqLinkNews = pd.read_csv('/Users/Cylita/Desktop/insight-ds-project_news-link/data/processed/Cleaned_Split_RawNews.csv')
trimmed_full = UniqLinkNews[['author','date', 'title', 'maintext', 'mainurl', 'simlinks', 'sim1']]

In [16]:
#Combining the above two dataframes
raw_val = pd.concat([trimmed_full, trimmed_sim], axis=0, ignore_index = True, sort=True)

In [17]:
#For the above combined dataframe - creating combined text and cleaning 
#Combing the news story headlines with the maintext for the stories
full_text = raw_val["title"].map(str)+ '. ' + raw_val["maintext"]

#Preprocessing all text to conduct tl-dlf cosine similarity analysis
normfull = TN.normalize_NewsText(full_text)

KeyboardInterrupt: 

In [146]:
#Adding in cleaned text to dataframe
raw_val['full_text'] = full_text
raw_val['cleaned_text'] = normfull

In [149]:
#Exporting cleaned and processed text to a corpus document
raw_val.to_csv("Val_Corpus.csv")

In [18]:
#Reading back in the new dataframe to work from
val_corpus = pd.read_csv("Val_Corpus.csv")